Libraries Required

In [ ]:
import pandas as pd
import os 
import numpy as np
import glob
import subprocess,openpyxl
from openpyxl import load_workbook
import re
import xlsxwriter
import zipfile

Step 1.1 Below code will help to zip all the files under one folder

In [ ]:
def zip_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        # define the file extension
        if file_name.endswith(".xml"): 
            file_name_1 = file_name.split('.')[0]
            file_path = os.path.join(folder_path, file_name)
            zip_file_path = os.path.join(folder_path, f"{file_name_1}.zip")
 
            with zipfile.ZipFile(zip_file_path, 'w',zipfile.ZIP_DEFLATED) as zip_file:
                zip_file.write(file_path, file_name)
 
if __name__ == "__main__":
    folder_path = "Path"  # Replace with your actual folder path
    zip_files_in_folder(folder_path)

Step 1.2 To process the files using connector use below code.

In [ ]:
# All files & Folder for this particular script must be kept under below path
folder = 'Path' # Enter folder location
jarFile = f'{folder}Connector_Name.jar'
#Download Metadata template and put it inside another folder with folder name as "excel"  
base = f'{folder}excel\\Template_Name.xlsx'
# Mention which type of file format trying to read like below have .zip extension
file_list = glob.glob(folder + "/*.zip")
for i in range(len(file_list)):
    data = file_list[i].split('.')[0]
    wb=openpyxl.load_workbook(base)
    output_excel = f'{data}.xlsx'
    wb.save(output_excel)
    print(output_excel)
    try:
        cmd = f'cmd /c java -jar "{jarFile}" "{file_list[i]}" "{output_excel}"'
        subprocess.run(cmd,shell=True,check=True)
    except Exception as e:
        print(f'{data} : {e}')

Step 2.1 To handle duplicate in one column while merging use below code 

In [ ]:
all_data = pd.DataFrame()
# Specify the folder containing your Excel files
folder_path = "Path"
file_list = glob.glob(folder_path + "/*.xlsx")
count = 0
for i in file_list: 
    print(i)
    # Define the sheet name if any
    df = pd.read_excel(i, header=0, sheet_name='Attribute Level Metadata')
    all_data=all_data.drop_duplicates()
    df = df.drop_duplicates()
    if 'REPORT_NAME' in all_data.columns:
        # Update report names with the appropriate suffix for instances in the second file
        report_names_set = set(all_data['REPORT_NAME'])
    # Update report names with the appropriate suffix
        for index, row in df.iterrows():
            report_name = row['REPORT_NAME']
            if report_name in report_names_set:
                df.at[index, 'REPORT_NAME'] += f"_1"
                count=2
                while True:
                    if df.at[index, 'REPORT_NAME'] in report_names_set:
                        df.at[index, 'REPORT_NAME']= df.at[index, 'REPORT_NAME'][:-2]+f"_{count}"
                        print(df.at[index, 'REPORT_NAME'])
                        count+=1
                    else:
                        break
    all_data = all_data._append(df, ignore_index=True)
print('outside loop')
output_file_path = f"{folder_path}"
rows_per_file = 1000000
number_of_files = ((len(all_data)//rows_per_file))+1
start_index=0
end_index = rows_per_file
for i in range(number_of_files):
    filepart = f'{folder_path}\\final_{i}.xlsx'
    writer = pd.ExcelWriter(filepart)
    df_mod = all_data.iloc[start_index:end_index]
    df_mod.to_excel(writer, index=False, sheet_name='sheet')
    start_index = end_index
    end_index = end_index + rows_per_file
    writer.close()
print("Data merged and saved to", output_file_path)

Step 3.1 Below code will help you to perform package splitting

In [ ]:
# Enter the folder Name in which your output will be generated
package='Package_Name'
# Give the folder location of parent dir
parent = 'Path'
# Give excel location which you want to read usually the generated excel from previous code
data_df = pd.read_excel(r"Path\\Excel_Name.xlsx",header=0)
path=os.path.join(parent,package)
os.makedirs(path, exist_ok=True)
try:
    # Define the column name according to which grouping will perfomed
    grouped_df = data_df.groupby('ATTRIBUTE_AGGREGATION')
except Exception as e:
    pass
for data in grouped_df:
    try:
        print(data[0])
        grouped_df.get_group(data[0]).to_excel(f"{parent}{package}\\"+data[0]+".xlsx",index=False)
    except Exception as e:
        value = re.findall(r"'([^']*)'",data[0])
        if len(value)>0:
            print(value[-1])
            grouped_df.get_group(data[0]).to_excel(f"{parent}{package}\\"+value[-1]+".xlsx",index=False)
        else:
            if '/' in data[0]:
                value_1 = data[0].replace('/','_')
                print(value_1)
                grouped_df.get_group(data[0]).to_excel(f"{parent}{package}\\"+value_1+".xlsx",index=False)

Step 4.1 If the file name of both excel is same it will merge the excel and save it in Target directory.

In [ ]:
# Give both the folder location
folder_path = "Folder_1"
file_list = glob.glob(folder_path + "/*.xlsx")
folder_path_1 = "Folder_2"
# Define Output folder name below
file_list_1 = glob.glob(folder_path_1 + "/*.xlsx")
count=0
for i in file_list:
    data1 = i.split('\\')[-1]
    data1 = re.sub('.xlsx','',data1)
    for j in file_list_1:
        data2 = j.split('\\')[-1]
        data2 = re.sub('.xlsx','',data2)
        if data1==data2:
            count+=1
            try:
                df1 = pd.read_excel(i, header=0)
                df2 = pd.read_excel(j, header=0)
                df3 = df1._append(df2, ignore_index=True)
                df3= df3.drop_duplicates()
                # Below create a folder manually and define the path
                path = f"Folder Name\\{data2}.xlsx"
                df3.to_excel(path, sheet_name='Sheet1', index=False)
            except Exception as e:
                print(e)
            break
print(count)

Step 5.1 To extract unique report count from multiple packages/metadata use below code

In [ ]:
df2 = pd.DataFrame(columns=['Package','Report_Count','Desc_Count'])
#Give the source folder location
file_list_1 = glob.glob(r'Folder_Location' + "/*.xlsx")
for i in file_list_1:
    print(i)
    file_name = i.split('\\')[-1]
    file_name = re.sub('.xlsx','',file_name)
    try:
        df = pd.read_excel(i, header=0)
    except Exception as e:
        print(e)
    report_name_count=len(df['REPORT_NAME'].unique())
    report_desc_count=len(df['REPORT_DESCRIPTION'].unique())
    df2 = df2._append({'Package':file_name,'Report_Count':report_name_count,'Desc_Count':report_desc_count},ignore_index=True)
df2.to_excel(r'Target_Excel_location\\Excel_Name.xlsx',index=False) # define the target folder and file location

Step 5.2 To get report harmonization sheet details use below code

In [ ]:
df2 = pd.DataFrame(columns=['Package','Functional Area','Report_Count','Required Report'])
#Give the source folder location
file_list_1 = glob.glob('Folder_Location' + "/*.xlsm")
for i in file_list_1:
    print(i)
    file_name = i.split('\\')[-1]
    file_name = re.sub('.xlsm','',file_name)
    wb = openpyxl.load_workbook(i,read_only=True,keep_vba=True)
    sheet = wb['Summary']
    c7 = sheet['C7'].value
    c9 = sheet['C9'].value
    c4 = sheet['C4'].value
    #c4 = sheet['C4'].value
    df2 = df2._append({'Package':file_name,'Functional Area':c4,'Report_Count':c7,'Required Report':c9},ignore_index=True)
df2.to_excel(r'Target_Excel_location\\Excel_Name.xlsx',index=False) # define the target folder and file location

In [ ]:
df1=df_2.groupby(['ATTRIBUTE_AGGREGATION','ATTRIBUTE_TYPE'])['REPORT_NAME'].nunique().reset_index(name='DistinctReportCount')